In [228]:
import os
import json
import pandas as pd
import traceback

In [229]:
from langchain_openai import ChatOpenAI

In [230]:
from dotenv import load_dotenv

load_dotenv()

True

In [231]:
Key = os.getenv("OPENAI_API_KEY")

In [232]:
llm = ChatOpenAI(openai_api_key= Key, model_name = "gpt-3.5-turbo", temperature = 0.5)

In [233]:
llm

ChatOpenAI(profile={'max_input_tokens': 16385, 'max_output_tokens': 4096, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': False, 'structured_output': False, 'image_url_inputs': False, 'pdf_inputs': False, 'pdf_tool_message': False, 'image_tool_message': False, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x000001970A9685F0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001970A969E50>, root_client=<openai.OpenAI object at 0x000001970A9688C0>, root_async_client=<openai.AsyncOpenAI object at 0x000001970A9695E0>, temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [234]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda
import PyPDF2

In [235]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [236]:
Template = """ 
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [237]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    template= Template
)

In [238]:
quiz_chain = quiz_generation_prompt | llm ## New way of forming chain

In [239]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [240]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=['subject', 'quiz'],
    template= TEMPLATE2
)

In [241]:
review_chain = quiz_evaluation_prompt | llm

In [242]:
from langchain_core.runnables import RunnableLambda

def run_quiz_and_review(inputs):
    # 1) generate quiz
    quiz = quiz_chain.invoke(inputs)
    # 2) review quiz
    review = review_chain.invoke(
        {
            "subject": inputs["subject"],
            "quiz": quiz,
        }
    )
    # 3) return both
    return {"quiz": quiz, "review": review}

generate_evaluate_chain = RunnableLambda(run_quiz_and_review)


In [243]:
file_path = r"D:\Projects\mcqGenerator\data.txt"

In [244]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [245]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [246]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [247]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [248]:
from langchain_community.callbacks.manager import get_openai_callback

with get_openai_callback() as cb:
    response = generate_evaluate_chain.invoke(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON),
        }
    )

In [249]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1715
Prompt Tokens:1361
Completion Tokens:354
Total Cost:0.0012115


In [250]:
response

{'quiz': AIMessage(content='\n{\n"1": {\n"mcq": "What is biology?",\n"options": {\n"a": "The study of rocks",\n"b": "The scientific study of life",\n"c": "The study of stars",\n"d": "The study of economics"\n},\n"correct": "b"\n},\n"2": {\n"mcq": "What is the main theme that explains the unity and diversity of life?",\n"options": {\n"a": "Evolution",\n"b": "Chemistry",\n"c": "Physics",\n"d": "Mathematics"\n},\n"correct": "a"\n},\n"3": {\n"mcq": "What is the basic unit of life?",\n"options": {\n"a": "Atoms",\n"b": "Molecules",\n"c": "Cells",\n"d": "Organs"\n},\n"correct": "c"\n},\n"4": {\n"mcq": "What process allows organisms to move, grow, and reproduce?",\n"options": {\n"a": "Photosynthesis",\n"b": "Respiration",\n"c": "Energy processing",\n"d": "Digestion"\n},\n"correct": "c"\n},\n"5": {\n"mcq": "What is the scientific method used by biologists to study the world around them?",\n"options": {\n"a": "Guessing",\n"b": "Intuition",\n"c": "Trial and error",\n"d": "Making observations, pos

In [263]:
quiz = response.get("quiz").content

In [265]:
quiz = json.loads(quiz)

In [266]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [267]:
quiz_table_data

[{'MCQ': 'What is biology?',
  'Choices': 'a: The study of rocks | b: The scientific study of life | c: The study of stars | d: The study of economics',
  'Correct': 'b'},
 {'MCQ': 'What is the main theme that explains the unity and diversity of life?',
  'Choices': 'a: Evolution | b: Chemistry | c: Physics | d: Mathematics',
  'Correct': 'a'},
 {'MCQ': 'What is the basic unit of life?',
  'Choices': 'a: Atoms | b: Molecules | c: Cells | d: Organs',
  'Correct': 'c'},
 {'MCQ': 'What process allows organisms to move, grow, and reproduce?',
  'Choices': 'a: Photosynthesis | b: Respiration | c: Energy processing | d: Digestion',
  'Correct': 'c'},
 {'MCQ': 'What is the scientific method used by biologists to study the world around them?',
  'Choices': 'a: Guessing | b: Intuition | c: Trial and error | d: Making observations, posing questions, generating hypotheses, performing experiments, and forming conclusions',
  'Correct': 'd'}]

In [269]:
quiz = pd.DataFrame(quiz_table_data)

In [270]:
quiz.to_csv("biology_quiz.csv", index = False)